In [2]:
import pandas as pd
from collections import defaultdict
import json

In [20]:
df = pd.read_csv('test.csv')

In [33]:
def get_meta_by_hscode(hscode):
    hscode = str(int(hscode))
    meta = {}
    if hscode.startswith('8702'):
        meta['type'] = 'public-vehicle'
    elif hscode.startswith('8703'):
        meta['type'] = 'four-wheelers'
    elif hscode.startswith('8711'):
        meta['type'] = 'two-wheeler'
    return meta


In [46]:
new_df = pd.DataFrame(columns=['hscode', 'year', 'month', 'description', 'type', 'status', 'fuel', 'quantity', 'value', 'revenue'])

for year, ydf in df.groupby('year'):
    for month, mdf in ydf.groupby('month'):
        for i, row in mdf.iterrows():
            meta = {}
            description = row['description'].lower()
            if 'unassembled' in description:
                meta['status'] = 'unassembled'
            else:
                meta['status'] = 'assembled'
            if 'electric' in description:
                meta['fuel'] = 'electric'
            if 'hybrid' in description or (('electric' in description or 'elec' in description) and 'petrol' in description):
                meta['fuel'] = 'hybrid'
            if not meta.get('fuel', None):
                meta['fuel'] = 'defacto'
            if any([word in description for word in ['bike', 'moped', 'scooter', 'cycle', 'motorcycle']]):
                meta['type'] = 'two-wheeler'
            elif any([word in description for word in ['rickshaw', 'three wheeler','3 wheeler', 'tempo']]):
                meta['type'] = 'three-wheeler'
            elif any([word in description for word in ['car', 'jeep', 'van']]):
                meta['type'] = 'four-wheeler'
            elif any([word in description.replace('combustion', '') for word in ['bus', 'mini-bus', 'micro-bus']]):
                meta['type'] = 'public-vehicle'
            elif any([word in description for word in ['ambulance', 'sab-bahan', 'sabbahan', 'sab bahan']]):
                meta['type'] = 'ambulance-sabbahan'
            
            if not meta.get('type', None):
                hscode_meta = get_meta_by_hscode(row['hscode'])
                meta = {**meta, **hscode_meta}

            new_row_data = pd.DataFrame({
                'year': year,
                'month': month,
                'hscode': int(row['hscode']),
                'description':row['description'],
                'type': meta['type'],
                'status': meta['status'],
                'fuel': meta['fuel'],
                'quantity': row['quantity'],
                'value': row['value'],
                'revenue': row['revenue']
            }, index=[0])
            new_df = pd.concat([new_df, new_row_data], ignore_index=True)

/tmp/ipykernel_7399/3270273699.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, new_row_data], ignore_index=True)


In [44]:
new_df

,hscode,year,month,description,type,status,fuel,quantity,value,revenue
0,87032110.0,2074,4,Auto rickshaw (three wheeler) <=1000 cc,three-wheeler,assembled,defacto,1109.0,164691.660000,205981.000
1,87032190.0,2074,4,Other vehicles of a capacity upto 1000cc .,four-wheelers,assembled,defacto,187.0,62871.590000,139317.440
2,87032200.0,2074,4,Vehicles of a cylinder capacity exceeding 00...,four-wheelers,assembled,defacto,568.0,359065.420000,789927.230
3,87032300.0,2074,4,Vehicles of a cylinder capacity exceeding 50...,four-wheelers,assembled,defacto,185.0,199491.690000,428537.500
4,87033100.0,2074,4,Vehicles with diesel engine of cylinder ca ac...,four-wheelers,assembled,defacto,58.0,31515.280000,69682.270
...,...,...,...,...,...,...,...,...,...,...
2332,87113099.0,2080,10,Motorcycles with reciprocating piston enginee ...,two-wheeler,assembled,defacto,0.0,0.000000,0.000
2333,87114000.0,2080,10,Motorcycle With piston engine of exceeding 500...,two-wheeler,assembled,defacto,2.0,535.543180,0.565
2334,87115000.0,2080,10,Motorcycle with piston engine exceeding 800 CC,two-wheeler,assembled,defacto,0.0,0.000000,0.000
2335,87116010.0,2080,10,Cycle with electric motor for propulsion,two-wheeler,assembled,electric,385.0,8257.120812,2007.613


In [47]:
new_df.to_csv('final.csv', index=False)